# Сбор датасетов из таблиц википедии

Соберем три различных датасета из англоязычных таблиц википедии:

1) Коэффициенты УЕФА - https://en.wikipedia.org/wiki/UEFA_coefficient

2) Список самых продаваемых видеоигр - https://en.wikipedia.org/wiki/List_of_best-selling_video_games

3) Список языков программирования, возникших с 2010 года - https://en.wikipedia.org/wiki/Timeline_of_programming_languages

## Необходимые функции

In [1]:
import math
import re
import requests

import lxml.html as lh
import numpy as np
import pandas as pd
import seaborn as sns


sns.set()


def get_col_names(col_names):
    """Возвращает список именом столбцов. В таблице может быть 
    заголовок, состоящий из более чем одной строки. В таком случае
    `pandas.read_html()` возвращает заголовок таблицы в формате 
    `pandas.core.index.MultiIndex`, где индексами являются кортежи 
    строк. Данная функция объединяет уникальные элементы таких кортежей 
    в строки через '/'. Например кортеж `('Rank', 'Rank')` заменяется
    строкой `'Rank'`, а кортеж `('Length', 'Electrified')` строкой 
    `'Length/Electrified'`. Если элементом `col_names` является строка,
    то она остается без изменений.
    
    Параметры:
        col_names: последовательность имен столбцов в `pandas.DataFrame`
    
    Возвращает:
        список строк
    
    Исключения:
        ValueError, если элемент `col_names` -- не кортеж и не строка. 
    """
    names = []
    for n in col_names:
        if isinstance(n, tuple):
            sn = ''
            for i, w in enumerate(n):
                if w not in n[:i]:
                    sn += '/' + w
            names.append(sn[1:])
        elif isinstance(n, (str, int)):
            names.append(n)
        else:
            raise ValueError(
                "Unsupported column name type {}\n"
                "Only types `str`, `int` and `Tuple[str]` "
                "are supported.\n"
                "column name: {}".format(type(n), n)
            )
    return names


def remove_excess_info(s):
    """Возвращает строку, полученную из `s` удалением из нее квадратных
    и круглых скобок с их содержимым, а также удалением символа *.
    
    Параметры:
        s: Любой объект. Если не строка, объект возвращается без изменений.
    
    Возвращает:
        строка
    """
    if isinstance(s, str):
        s = re.sub(r'((\[[^\]]*])|(\*)|(~)|(%)|(\([^\)]*\)))', '', s)
        s = re.sub(' +', ' ', s)
        s = s.strip()
    return s

        
def tofloat(x):
    if isinstance(x, str) and x in '—–' \
            or isinstance(x, float) and math.isnan(x):
        return float('nan')
    if isinstance(x, str):
        x = re.sub('[, ]+', '', x)
    try:
        return float(x)
    except ValueError:
        raise ValueError(
            "Cannot convert {} to `float`\nx=={}".format(
                type(x), x))
    


def get_tables_from_html(url):
    """Возвращает список таблиц найденных на веб-странице с адресом `url`.
    
    Параметры:
        url: строка
    
    Возвращает:
        список объектов класса `pandas.DataFrame`
    """
    # Скачиваем веб-страницу
    page = requests.get(url)
    # Преобразуем скачанную страницу в html документ.
    doc = lh.fromstring(page.content.decode('utf-8'))
    # Извлекаем из документа все таблицы
    tables = doc.xpath('//table')
    dfs = []
    for table in tables:
        df = pd.read_html(lh.tostring(table))[0]
        # Следующие 2 строчки кода решают проблему многострочных заголовков
        col_names = get_col_names(df.columns)
        df.columns = col_names
        dfs.append(df)
    return dfs


def prepare_wiki_table(df, int_cols):
    df = df.applymap(remove_excess_info)
    for col in int_cols:
        df[col] = df[col].map(tofloat)
    return df

## Коэффициенты УЕФА

In [2]:
url = "https://en.wikipedia.org/wiki/UEFA_coefficient"
dfs = get_tables_from_html(url)

In [3]:
len(dfs)

63

In [4]:
dfs[10].head()

,Ranking/2020,Ranking/2019,Ranking/Mvmt,"Member association(L: League, C: Cup, LC: League cup1)",Coefficient/2015–16,Coefficient/2016–17,Coefficient/2017–18,Coefficient/2018–19,Coefficient/2019–20,Coefficient/Total,Teams,Places in 2021–22 season/CL,Places in 2021–22 season/EL,Places in 2021–22 season/ECL,Places in 2021–22 season/Total
0,1,1,—,"Spain (L, C)",23.928,20.142,19.714,19.571,14.357,97.712,7/7,4,2,1,7
1,2,2,—,"England (L, C, LC1)",14.250,14.928,20.071,22.642,14.571,86.462,7/7,4,2,1,7
2,3,4,+1,"Germany (L, C)",16.428,14.571,9.857,15.214,12.428,68.498,6/7,4,2,1,7
3,4,3,–1,"Italy (L, C)",11.500,14.250,17.333,12.642,10.785,66.510,5/7,4,2,1,7
4,5,5,—,"France (L, C, LC1)",11.083,14.416,11.500,10.583,8.500,56.082,2/6,3,2,1,6


In [5]:
UEFA_coefficient = prepare_wiki_table(dfs[10],['Coefficient/Total'])

In [6]:
UEFA_coefficient = UEFA_coefficient.rename({'Member association(L: League, C: Cup, LC: League cup1)': 'Country'}, axis=1)

In [7]:
UEFA_coefficient = UEFA_coefficient.set_index('Country')

In [8]:
UEFA_coefficient.shape

(55, 14)

In [9]:
UEFA_coefficient.describe()

,Ranking/2020,Ranking/2019,Coefficient/2017–18,Coefficient/2018–19,Coefficient/2019–20,Coefficient/Total,Places in 2021–22 season/CL,Places in 2021–22 season/EL,Places in 2021–22 season/ECL,Places in 2021–22 season/Total
count,55.00000,55.00000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000
mean,28.00000,28.00000,4.481782,4.573509,3.966036,21.818800,1.436364,0.363636,3.872727,4.309091
std,16.02082,16.02082,4.671718,4.598842,3.441113,20.959631,0.897809,0.648749,4.891620,1.086495
min,1.00000,1.00000,0.000000,0.375000,0.000000,0.666000,0.000000,0.000000,1.000000,1.000000
25%,14.50000,14.50000,1.125000,1.500000,1.500000,6.725000,1.000000,0.000000,2.000000,4.000000
50%,28.00000,28.00000,2.900000,3.125000,3.000000,16.625000,1.000000,0.000000,3.000000,4.000000
75%,41.50000,41.50000,5.562500,5.875000,4.833000,27.025000,2.000000,1.000000,3.000000,5.000000
max,55.00000,55.00000,20.071000,22.642000,14.571000,97.712000,4.000000,2.000000,23.000000,7.000000


In [10]:
UEFA_coefficient

,Ranking/2020,Ranking/2019,Ranking/Mvmt,Coefficient/2015–16,Coefficient/2016–17,Coefficient/2017–18,Coefficient/2018–19,Coefficient/2019–20,Coefficient/Total,Teams,Places in 2021–22 season/CL,Places in 2021–22 season/EL,Places in 2021–22 season/ECL,Places in 2021–22 season/Total
Country,,,,,,,,,,,,,,
Spain,1,1,—,23.928,20.142,19.714,19.571,14.357,97.712,7/7,4,2,1,7
England,2,2,—,14.250,14.928,20.071,22.642,14.571,86.462,7/7,4,2,1,7
Germany,3,4,+1,16.428,14.571,9.857,15.214,12.428,68.498,6/7,4,2,1,7
Italy,4,3,–1,11.500,14.250,17.333,12.642,10.785,66.510,5/7,4,2,1,7
France,5,5,—,11.083,14.416,11.500,10.583,8.500,56.082,2/6,3,2,1,6
Portugal,6,7,+1,10.500,8.083,9.666,10.900,9.700,48.849,4/5,3,1,2,6
Russia,7,6,–1,11.500,9.200,12.600,7.583,4.666,45.549,0/6,2,1,2,5
Belgium,8,8,—,7.400,12.500,2.600,7.800,7.200,37.500,2/5,2,1,2,5
Netherlands,9,11,+2,5.750,9.100,2.900,8.600,8.800,35.150,2/5,2,1,2,5


## Список самых продаваемых видеоигр

In [11]:
url = "https://en.wikipedia.org/wiki/List_of_best-selling_video_games"
dfs = get_tables_from_html(url)

In [12]:
len(dfs)

4

In [13]:
dfs[1].head()

,Title,Sales,Platform(s),Initial release date,Developer(s)[a],Publisher(s)[a],Bundled with a console?,Ref.
0,Minecraft,"180,000,000[b]",Multi-platform,"November 18, 2011[c]",Mojang,Xbox Game Studios,NaN,[3]
1,Tetris,"170,000,000[b]",Multi-platform,"June 6, 1984",Elektronorgtechnica,Various[d],NaN,[5][e]
2,Grand Theft Auto V,115000000,Multi-platform,"September 17, 2013",Rockstar North,Rockstar Games,NaN,[9]
3,Wii Sports,82880000,Wii,"November 19, 2006",Nintendo EAD,Nintendo,NaN,[10]
4,PlayerUnknown's Battlegrounds,50000000,Multi-platform,"December 20, 2017",PUBG Corporation,PUBG Corporation,NaN,[11]


In [14]:
VideoGames = prepare_wiki_table(dfs[1],['Sales'])

In [15]:
VideoGames = VideoGames.set_index('Title')

In [16]:
VideoGames = VideoGames.drop(['Bundled with a console?','Ref.'],axis=1)

In [17]:
VideoGames.shape

(51, 5)

In [18]:
VideoGames.describe()

,Sales
count,5.100000e+01
mean,3.441306e+07
std,3.321759e+07
min,1.600000e+07
25%,2.000000e+07
50%,2.473000e+07
75%,3.000000e+07
max,1.800000e+08


In [19]:
VideoGames

,Sales,Platform(s),Initial release date,Developer(s)[a],Publisher(s)[a]
Title,,,,,
Minecraft,180000000.0,Multi-platform,"November 18, 2011",Mojang,Xbox Game Studios
Tetris,170000000.0,Multi-platform,"June 6, 1984",Elektronorgtechnica,Various
Grand Theft Auto V,115000000.0,Multi-platform,"September 17, 2013",Rockstar North,Rockstar Games
Wii Sports,82880000.0,Wii,"November 19, 2006",Nintendo EAD,Nintendo
PlayerUnknown's Battlegrounds,50000000.0,Multi-platform,"December 20, 2017",PUBG Corporation,PUBG Corporation
Super Mario Bros.,48240000.0,Multi-platform,"September 13, 1985",Nintendo,Nintendo
Pokémon Red/Green/Blue/Yellow,47520000.0,Multi-platform,"February 27, 1996",Game Freak,Nintendo
Wii Fit and Wii Fit Plus,43800000.0,Wii,"December 1, 2007",Nintendo EAD,Nintendo
Mario Kart Wii,37240000.0,Wii,"April 10, 2008",Nintendo EAD,Nintendo


## Список языков программирования, возникших с 2010 года 

In [20]:
url = "https://en.wikipedia.org/wiki/Timeline_of_programming_languages"
dfs = get_tables_from_html(url)

In [21]:
len(dfs)

13

In [22]:
dfs[11].head()

,Year,Name,"Chief developer, company",Predecessor(s)
0,2010,Rust,"Graydon Hoare, Mozilla","Alef, C++, Camlp4, Erlang, Hermes, Limbo, Napi..."
1,2011,C11,C11 ISO/IEC 9899:2011,NaN
2,2011,Ceylon,"Gavin King, Red Hat",Java
3,2011,Dart,Google,"Java, JavaScript, CoffeeScript, Go"
4,2011,C++11,C++ ISO/IEC 14882:2011,"C++, Standard C, C"


In [23]:
ProgLang = dfs[11]

In [24]:
ProgLang = ProgLang.query("Name != 'Name'")

In [25]:
ProgLang = ProgLang.set_index('Name')

In [26]:
ProgLang.shape

(31, 3)

In [27]:
ProgLang.describe()

,Year,"Chief developer, company",Predecessor(s)
count,31,30,25
unique,9,30,23
top,2011,JetBrains,"C++, Standard C, C"
freq,8,1,3


In [28]:
ProgLang

,Year,"Chief developer, company",Predecessor(s)
Name,,,
Rust,2010,"Graydon Hoare, Mozilla","Alef, C++, Camlp4, Erlang, Hermes, Limbo, Napi..."
C11,2011,C11 ISO/IEC 9899:2011,NaN
Ceylon,2011,"Gavin King, Red Hat",Java
Dart,2011,Google,"Java, JavaScript, CoffeeScript, Go"
C++11,2011,C++ ISO/IEC 14882:2011,"C++, Standard C, C"
Kotlin,2011,JetBrains,"Java, Scala, Groovy, C#, Gosu"
Red,2011,Nenad Rakocevic,"Rebol, Scala, Lua"
MonkeyX,2011,Mark Sibly,NaN
Opa,2011,MLstate,"OCaml, Erlang, JavaScript"
